Analytics & Data Science

Universidad de Antioquia - ML2

Feb 2024

Natalia López Grisale CC. 1040048893
Tatiana García Zuluaga CC. 1017198484
Melissa Ortega Alzate CC.1036964792

# Libraries

In [13]:
# Natural Language Toolkit
import nltk         
from nltk.tokenize import sent_tokenize         # Tokenization is used to divide the text into words, sentences or other units.

# Calculate distances
from sklearn.metrics.pairwise import cosine_similarity

# Open Source library to build embeddings
#! pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer


# Functions

In [14]:
# Define function to read the text file
def read_text_from_file(file_name):
    """
    Read text data from a file.
    Args:
        file_name (str): The name of the file to read.
    Returns:
        str: The text read from the file.
    """
    with open(file_name, 'r', encoding='iso-8859-1') as file:
        text = file.read()
    return text

In [15]:
# Define function to split the text into a number of desired sentences
def split_text(text, sentences_per_fragment):
    """
    Split text into fragments based on a specified number of lines per fragment.

    Args:
        text (str): The text to be split into fragments.
        lines_per_fragment (int): The number of lines per fragment.

    Returns:
        list of str: List of fragments, where each fragment contains the specified number of lines.
    """
    lines = text.split('.')
    fragments = []
    
    for i in range(0, len(lines), sentences_per_fragment):
        fragment = '.'.join(lines[i: i+sentences_per_fragment])
        fragments.append(fragment)
    
    return fragments

# 1. In your own words, describe what vector embeddings are and what they are useful for.

The embeddings are a numerical representation not only of language but also of its context and semantics. 

They are fixed-dimensional vectors constructed from texts so that ML models can understand audio, text, image, video instructions, etc. Embeddings are, therefore, a representation in a large dimensional space with the best possible meaning of context and semantics. Since embeddings are vectors, they can be manipulated with all traditional linear algebra techniques.

These embeddings are useful in many areas, and thanks to them, recommendation systems (YouTube, Netflix), semantic search engines (YouTube), translators, ChatGPT, text classifiers, and in general, all language understanding AI models have been built.

# 2. What do you think is the best distance criterion to estimate how far two embeddings ...

# 3. Let us build a Q&A (question answering) system!

### a. Pick a text

The text was taken and edited from: https://aws.amazon.com/what-is/machine-learning/?nc1=h_ls

In [16]:
# Load the text using the predefined function
text = read_text_from_file('suggested_text/prueba.txt')

# Text description
print("The length of the text is", len(text), "characters\n")
print("The text up to character 342 is:\n", text[:342])

The length of the text is 14044 characters

The text up to character 342 is:
 Machine learning is the science of developing algorithms and statistical models that computer systems use to perform tasks without explicit instructions, relying on patterns and inference instead. Computer systems use machine learning algorithms to process large quantities of historical data and identify data patterns. This allows them to p


### b. Split that text into meaningful chunks/pieces.

In [17]:
# Split the text into fragments
fragments = split_text(text, 2)

# Print the fragments
for i, fragment in enumerate(fragments):
    print(f"Fragment {i+1}: {fragment}\n")

# Print the total number of fragments
print(f"\nThe text was divided into {i+1} fragments")

Fragment 1: Machine learning is the science of developing algorithms and statistical models that computer systems use to perform tasks without explicit instructions, relying on patterns and inference instead. Computer systems use machine learning algorithms to process large quantities of historical data and identify data patterns

Fragment 2:  This allows them to predict outcomes more accurately from a given input data set. For example, data scientists could train a medical application to diagnose cancer from x-ray images by storing millions of scanned images and the corresponding diagnoses

Fragment 3: 
Machine learning helps businesses by driving growth, unlocking new revenue streams, and solving challenging problems. Data is the critical driving force behind business decision-making but traditionally, companies have used data from various sources, like customer feedback, employees, and finance

Fragment 4:  Machine learning research automates and optimizes this process. By using sof

### c. Implement the embedding generation logic

In [18]:
# Instantiate the pretrained model
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')

# Generate embeddings for each fragment
embeddings = model.encode(fragments)

# Sample data
print("Number of embeddings:", len(embeddings))
print(f"\nEmbedding for fragment 1:\n {embeddings[0]}:")

Number of embeddings: 66

Embedding for fragment 1:
 [-1.48474304e-02  7.25356862e-02  2.91582360e-03 -4.75783236e-02
 -7.07611069e-02 -4.08746339e-02 -2.42115725e-02  8.66363771e-05
 -8.34782571e-02  1.79940257e-02  9.69681889e-02  1.69683918e-01
 -7.51287714e-02  1.85720757e-01  5.01122735e-02 -2.02736065e-01
 -1.25303455e-02 -5.84237203e-02  4.42154631e-02  8.35931748e-02
 -3.54419723e-02 -7.57947415e-02  1.18775796e-02 -2.02463530e-02
 -3.27721760e-02 -9.49414372e-02 -4.07991782e-02 -1.10695690e-01
 -5.66377081e-02  1.07649215e-01 -5.01370691e-02 -1.66517437e-01
  1.41108125e-01  8.77800509e-02  1.66838113e-02 -3.44206765e-02
 -8.37415364e-03  9.68867838e-02  7.52341002e-02 -1.49951339e-01
  2.48079792e-01 -2.45287940e-02  7.27478713e-02 -4.47176807e-02
  2.09419243e-02  2.75891215e-01  6.25376925e-02 -1.25490114e-01
 -2.54110415e-02  1.12973325e-01  7.29606440e-03  9.62402895e-02
 -1.86455518e-01  5.36349453e-02 -1.10063732e-01 -9.58444029e-02
 -6.10179566e-02  1.28090963e-01 -3.8

In [19]:
# Print embeddings list characteristics
print("Variable type of embeddings:", type(embeddings))
print("Dimensions of embeddings:", embeddings.shape)

Variable type of embeddings: <class 'numpy.ndarray'>
Dimensions of embeddings: (66, 768)


- Each row corresponds to a fragment and each column to a dimension in the embedding space. In other words, each row in this matrix represents the embedding for a particular fragment.

- The pre-trained model is a sentence-transformers model: It maps sentences & paragraphs to a 768 dimensional dense vector space and can be used for tasks like clustering or semantic search.

### d. For every question, return a sorted list of the N pieces that relate the most to the question.

In [20]:
def get_answers(user_question, text_fragments, n=3):
    """
    Get the most related text fragments to a user question.

    Args:
        user_question (str): The question asked by the user.
        text_fragments (list): List of text fragments to compare with the user question.
        n (int, optional): Number of most related fragments to return. Defaults to 5.

    Returns:
        tuple: A tuple containing two lists:
               - similarities: List of tuples (fragment, similarity_score) for all fragments.
               - sorted_fragments: List of tuples (fragment, similarity_score) for the top N related fragments.
    """
    # Instantiate the pretrained model
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

    # Generate embedding for user question
    embedding_question = model.encode(user_question)

    # Initialize a list to store the similarity between the question and each fragment
    similarities = []

    # Generate embeddings for each text fragment and calculate cosine similarity
    for fragment in text_fragments:
        embedding_fragment = model.encode(fragment)
        similarity = cosine_similarity([embedding_question], [embedding_fragment], dense_output=True)[0][0]
        similarities.append((fragment, similarity))

    # Sort text fragments based on their similarity with the user question
    sorted_fragments = sorted(similarities, key=lambda x: x[1], reverse=True)

    # Return the top N most relevant fragments
    return sorted_fragments[:n]


In [21]:
# Using the function to find the most related fragments
user_question = "How does machine learning help businesses?"

questions = [
    "How does machine learning help businesses?",
    "How does machine learning improve financial services?",
    "What role does machine learning play in retail?",
    "How is machine learning used in media and entertainment?",
    "What are the strengths of supervised learning?",
    "How does unsupervised machine learning differ from supervised learning?",
    "What are some applications of machine learning in manufacturing?",
]

# Number of fragments
n = 2

In [22]:
# Iterating over the list of questions
for user_question in questions:
    related_fragments = get_answers(user_question, fragments, n=n)

    # Results
    print(f"============ Question: {user_question} =============")
    for fragment, similarity in related_fragments:
        print(f"{fragment}\n=> Similarity: {similarity}\n")


============ Question: How does machine learning help businesses? =============

Machine learning helps businesses by driving growth, unlocking new revenue streams, and solving challenging problems. Data is the critical driving force behind business decision-making but traditionally, companies have used data from various sources, like customer feedback, employees, and finance
=> Similarity: 0.8451827168464661


Letâs take a look at machine learning applications in some key industries:
Machine learning can support predictive maintenance, quality control, and innovative research in the manufacturing sector. Machine learning technology also helps companies improve logistical solutions, including assets, supply chain, and inventory management
=> Similarity: 0.7533918619155884

============ Question: How does machine learning improve financial services? =============

Machine learning helps businesses by driving growth, unlocking new revenue streams, and solving challenging problems. Data

# Bibliography



- Text: Source: AWS - What is Machine Learning?
- NLTK Tokenization API: Source: NLTK Tokenization Documentation
- Pretrained Model: Model: Hugging Face - sentence-transformers/all-MiniLM-L6-v2
- Cosine Similarity: Documentation: Scikit-learn - Cosine Similarity